In [ ]:
using JSON
using Distributed
using Plots
using LaTeXStrings
using ProgressMeter
theme(:default)
if nprocs() < length(Sys.cpu_info()) - 50
    addprocs(length(Sys.cpu_info()) - 50 - nprocs())
end
println("Using ",nprocs()," processes.")
include("./fermionise.jl")
include("./models.jl")
include("./quantuminfo.jl")

In [ ]:
function main(numBath)
    oneparthop_d0_arr = []
    impBathSflip_arr = []
    impBathCflip_arr = []
    oneparthop_01_arr = []
    basisStates = BasisStates(2 * (1 + numBath), 1 + numBath)
    U_arr = 1:1:20
    couplingsArray = [(1, 2, U, 1, -U, 0) for U in U_arr]
    computables = (
        ["hh" (1,2)],
        ["nh" (1,2)],
        ["hn" (1,2)],
        ["nn" (1,2)],
        ["+-+-" (1,2,4,3,)],
        ["+-+-" (2,1,3,4)],
        ["++--" (3,4,6,5)],
        ["++--" (5,6,4,3)],
        )
    entanglementMeasures = (
        ["VNE" (1, 2)],
        ["I2" ([1,2],[3,4])],
        ["I2" ([3,4],[5,6])],
        )
    
    (imp_hh, imp_nh, imp_hn, imp_nn, d0_spm, d0_smp, AB_cpm, BA_cmp, vne_d, I2_d0, I2_d1) = ComputeCorrelations(basisStates, TOSHamiltonianRSpace, couplingsArray, computables, entanglementMeasures)
    p = plot(palette = :Set1_9, thickness_scaling = 1.2, fontfamily="Roboto-Regular")
    plot!(U_arr, abs.(imp_hh), lw=3, label="h-occupancy")
    plot!(U_arr, abs.(imp_nn), lw=3, label="d-occupancy")
    plot!(U_arr, abs.(imp_hn + imp_nh), lw=3, label="1-occupancy")
    display(p)

    p = plot(palette = :Set1_9, thickness_scaling = 1.2, fontfamily="Roboto-Regular")
    plot!(U_arr, abs.(d0_spm .+ d0_smp), lw=3, label=L"\langle S_d^+S_0^-\rangle")
    plot!(U_arr, abs.(AB_cpm .+ BA_cmp), lw=3, label=L"\langle C_A^+ C_B^-\rangle")
    xlabel!(L"U/t_\perp")
    ylabel!("correlations")
    display(p)
    
    p = plot(palette = :Set1_9, thickness_scaling = 1.2, fontfamily="Roboto-Regular")
    plot!(U_arr, vne_d, lw=3, label=L"S_{EE}(d)")
    plot!(U_arr, I2_d1, lw=3, label=L"$I_2(A:B)$")
    xlabel!(L"U/t_\perp")
    ylabel!("correlations")
    display(p)
end

println(main(4))